Взял обученную mask-rcnn, заинференсили на всем тренировочном датасете, и соотнесли получившиеся полигоны с разметкой, и добавили это все к тренировочному датасету, получилось: 120к+100к(новых)=220к фото в датасете

https://drive.google.com/file/d/1IhDDTEmz_TxPN91i2Jcn_YxR25guY-Oq/view?usp=sharing

In [ ]:
!mkdir train_recognition
import zipfile
with zipfile.ZipFile('train_recognition_V3.zip') as zf:
    zf.extractall('train_recognition')

In [ ]:
!pip install -q opencv-python==4.5.4.60
!pip install -q hwb
!pip install -qU albumentations

In [ ]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
from google.colab.patches import cv2_imshow
import albumentations as albu

import numpy as np
import pandas as pd
import cv2
import os
import json
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
import random
import editdistance
from torch import autocast
from hwb import HandWrittenBlot

In [ ]:
train_df = pd.read_csv('/content/train_recognition/labels.csv')
train_data = [(train_df.file_name.values[i], train_df.text.values[i]) for i in range(len(train_df))]
train_data = [train_data[i] for i in range(len(train_data)) if 'D' not in train_data[i][1] and
              'Ы' not in train_data[i][1] and 'Ь' not in train_data[i][1] and 'z' not in train_data[i][1] and 'b' not in train_data[i][1] and 'f' not in train_data[i][1] and '*' not in train_data[i][1] and 'S' not in train_data[i][1] and 'V' not in train_data[i][1] and '}' not in train_data[i][1] and 'z' not in train_data[i][1] and
              'L' not in train_data[i][1] and '_' not in train_data[i][1] and 'h' not in train_data[i][1] and 'l' not in train_data[i][1] and 'r' not in train_data[i][1] and 's' not in train_data[i][1] and 't' not in train_data[i][1] and '<' not in train_data[i][1] and '>' not in train_data[i][1] and 'B' not in train_data[i][1] and 'O' not in train_data[i][1] and 'a' not in train_data[i][1] and 'c' not in train_data[i][1] and 'n' not in train_data[i][1] and 'Щ' not in train_data[i][1]]
random.seed(1)
random.shuffle(train_data)
print('train len', len(train_data))

split_coef = 0.8
train_len = int(len(train_data)*split_coef)

train_data_splitted = train_data[:train_len]
val_data_splitted = train_data[train_len:]

print('train len after split', len(train_data_splitted))
print('val len after split', len(val_data_splitted))


with open('train_recognition/train_labels_splitted.json', 'w') as f:
    json.dump(dict(train_data_splitted), f)
    
with open('train_recognition/val_labels_splitted.json', 'w') as f:
    json.dump(dict(val_data_splitted), f)

In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

config_json = {
    "alphabet": ' !|"\'()+,-./0123456789:;=?IN[]ЁАБВГДЕЖЗИКЛМНОПРСТУФХЦЧШЭЮЯабвгдежзийклмнопрстуфхцчшщъыьэюяё№',
    "save_dir": "/content/drive/MyDrive/NTO_AI/Hackathon/V3/weights",
    "num_epochs": 500,
    "image": {
        "width": 512,
        "height": 64
    },
    "train": {
        "root_path": "/content/train_recognition/images",
        "json_path": "train_recognition/train_labels_splitted.json",
        "batch_size": 64
    },
    "val": {
        "root_path": "/content/train_recognition/images",
        "json_path": "train_recognition/val_labels_splitted.json",
        "batch_size": 80
    }
}

In [ ]:
def collate_fn(batch):
    images, texts, enc_texts = zip(*batch)
    images = torch.stack(images, 0)
    text_lens = torch.LongTensor([len(text) for text in texts])
    enc_pad_texts = pad_sequence(enc_texts, batch_first=True, padding_value=0)
    return images, texts, enc_pad_texts, text_lens


def get_data_loader(
    transforms, json_path, root_path, tokenizer, batch_size, drop_last
):
    dataset = OCRDataset(json_path, root_path, tokenizer, transforms)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        collate_fn=collate_fn,
        batch_size=batch_size,
        num_workers=2,
    )
    return data_loader


class OCRDataset(Dataset):
    def __init__(self, json_path, root_path, tokenizer, transform=None):
        super().__init__()
        self.transform = transform
        with open(json_path, 'r') as f:
            data = json.load(f)
        self.data_len = len(data)

        self.img_paths = []
        self.texts = []
        for img_name, text in data.items():
            self.img_paths.append(os.path.join(root_path, img_name))
            self.texts.append(text)
        self.enc_texts = tokenizer.encode(self.texts)

    def __len__(self):
        return self.data_len

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        text = self.texts[idx]
        enc_text = torch.LongTensor(self.enc_texts[idx])
        image = cv2.imread(img_path)
        if self.transform is not None:
            image = self.transform(image)
        return image, text, enc_text


class AverageMeter:
    def __init__(self):
        self.reset()

    def reset(self):
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
CTC_BLANK = '<BLANK>'

def get_char_map(alphabet):
    char_map = {value: idx + 1 for (idx, value) in enumerate(alphabet)}
    char_map[CTC_BLANK] = 0
    return char_map


class Tokenizer:
    def __init__(self, alphabet):
        self.char_map = get_char_map(alphabet)
        self.rev_char_map = {val: key for key, val in self.char_map.items()}

    def encode(self, word_list):
        enc_words = []
        for word in word_list:
            enc_words.append(
                [self.char_map[char] for char in word])
        return enc_words

    def get_num_chars(self):
        return len(self.char_map)

    def decode(self, enc_word_list):
        dec_words = []
        for word in enc_word_list:
            word_chars = ''
            for idx, char_enc in enumerate(word):
                if (char_enc != self.char_map[CTC_BLANK]
                    and not (idx > 0 and char_enc == word[idx - 1])):

                    word_chars += self.rev_char_map[char_enc]
            dec_words.append(word_chars)
        return dec_words

In [ ]:
from albumentations.augmentations.transforms import JpegCompression	
class Normalize:
    def __call__(self, img):
        img = img.astype(np.float32) / 255
        return img


class ToTensor:
    def __call__(self, arr):
        arr = torch.from_numpy(arr)
        return arr


class MoveChannels:
    def __init__(self, to_channels_first=True):
        self.to_channels_first = to_channels_first

    def __call__(self, image):
        if self.to_channels_first:
            return np.moveaxis(image, -1, 0)
        else:
            return np.moveaxis(image, 0, -1)


class RandomWidth:
    def __init__(self, min_scale, max_scale):
        self.min_scale = min_scale
        self.max_scale = max_scale

    def __call__(self, image):
        random_scale = random.uniform(self.min_scale, self.max_scale)
        image = cv2.resize(image, (int(random_scale*image.shape[1]), image.shape[0]))
        if len(image.shape) == 2:
            image = np.expand_dims(image, axis=2)
        return image


class ImageResize:
    def __init__(self, height, width):
        self.height = height
        self.width = width

    def __call__(self, image):
        h, w, c = image.shape
        new_image = np.zeros((self.height, self.width, c), np.uint8)
        scale = self.height / h
        if int(w*scale) <= self.width:
            image = cv2.resize(image, (int(w*scale), self.height), interpolation=cv2.INTER_LINEAR)
            new_image[:, :image.shape[1], :] = image
        else:
            new_height = int(self.height * (self.width / int(w*scale)))
            image = cv2.resize(image, (self.width, new_height), interpolation=cv2.INTER_LINEAR)
            new_image[(self.height-new_height)//2:-((self.height-new_height)-(self.height-new_height)//2), :image.shape[1], :] = image
            
        return new_image

class HandwrittenBlots:
    def __init__(self, p=0.5):
        self.rectangle_info = {'x': (None, None),'y': (None, None),'h': (None, None),'w': (None, None)}
        self.blot_params = {'incline': (-10, 10),'intensivity': (0.1, 0.2),'transparency': (0.2, 0.3),'count': (1, 3)}
        self.blots = HandWrittenBlot(self.rectangle_info, self.blot_params)
        self.p = p

    def __call__(self, image):
        if random.random() < self.p:
            new_img = self.blots(image)
        else:
            new_img = image
        return new_img

class albu_aug:
    def __init__(self):
        self.aug_list = albu.Compose([
                                      albu.CoarseDropout(p=0.3),
                                      albu.Rotate(limit=5, p=0.6),
                                      albu.ToGray(p=0.2),
                                      albu.RandomBrightnessContrast(brightness_limit=0.1,
                                                                    contrast_limit=0.1, p=0.4),
                                      albu.CLAHE(clip_limit=4.0, tile_grid_size=(8, 8), p=0.2),
                                      albu.GridDistortion(p=0.2),
                                      albu.OpticalDistortion(p=0.2),
                                      albu.HueSaturationValue(hue_shift_limit=7, sat_shift_limit=7,
                                                              val_shift_limit=7, p=0.3),
                                      albu.RandomGamma(gamma_limit=(90, 110), p=0.3),
                                      albu.OneOf([
                                                  albu.Blur(blur_limit=5, p=0.1),
                                                  albu.MedianBlur(blur_limit=5, p=0.1)
                                      ]),
                                      albu.GaussNoise(var_limit=(20.0, 50.0), p=0.2),
                                      albu.RGBShift(r_shift_limit=7, g_shift_limit=7, b_shift_limit=7, p=0.3),
                                      albu.JpegCompression(quality_lower=75, p=0.15)])

    def __call__(self, image):
        image = self.aug_list(image=image)['image']
        image[:, :, 0] -= image[:, :, 0].min()
        image[:, :, 1] -= image[:, :, 1].min()
        image[:, :, 2] -= image[:, :, 2].min()
        return image


class Reduce_thickness:
    def __init__(self, p=1):
        self.p = p

    def __call__(self, image):
        if random.random() < self.p:
            kernel = np.ones((4,4), np.uint8)
            return cv2.dilate(image, kernel, iterations=1)
        else:
            return image


class Stretch:
    def __init__(self, p=1):
        self.p = p

    def __call__(self, image):
        if random.random() < self.p:
            stretch = (random.random() - 0.5)
            wStretched = max(int(image.shape[1] * (1 + stretch)), 1)
            img = cv2.resize(image, (wStretched, image.shape[0]))
        return image


def get_train_transforms(height, width):
    transforms = torchvision.transforms.Compose([
        HandwrittenBlots(p=0.25),
        albu_aug(),
        RandomWidth(0.85, 1.15),
        Reduce_thickness(p=0.07),
        Stretch(p=0.2),
        ImageResize(height, width),
        MoveChannels(to_channels_first=True),
        Normalize(),
        ToTensor()
    ])
    return transforms


def get_val_transforms(height, width):
    transforms = torchvision.transforms.Compose([
        ImageResize(height, width),
        MoveChannels(to_channels_first=True),
        Normalize(),
        ToTensor()
    ])
    return transforms

In [ ]:
def get_resnet34_new_backbone(pretrained=True):
    m = torchvision.models.resnet34(pretrained=pretrained)
    input_conv = nn.Conv2d(3, 64, 7, 1, 3)
    blocks = [input_conv, m.bn1, m.relu,
              m.maxpool, m.layer1, m.layer2, m.layer3]
    return nn.Sequential(*blocks)


class BiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout=0.1):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size, hidden_size, num_layers,
            dropout=dropout, batch_first=True, bidirectional=True)

    def forward(self, x):
        out, _ = self.lstm(x)
        return out


class CRNN(nn.Module):
    def __init__(self, number_class_symbols, time_feature_count=256, lstm_hidden=512, lstm_len=3, pretrained=True):
        super().__init__()
        self.feature_extractor = get_resnet34_new_backbone(pretrained=pretrained)
        self.avg_pool = nn.AdaptiveAvgPool2d((lstm_hidden, time_feature_count))
        self.bilstm = BiLSTM(lstm_hidden, lstm_hidden, lstm_len)
        self.classifier = nn.Sequential(
            nn.Linear(lstm_hidden*2, 368),
            nn.GELU(),
            nn.Dropout(0.1),
            nn.Linear(368, number_class_symbols))

    def forward(self, x):
        x = self.feature_extractor(x)
        b, c, h, w = x.size()
        x = x.view(b, c * h, w)
        x = self.avg_pool(x)
        x = x.transpose(1, 2)
        x = self.bilstm(x)
        x = self.classifier(x)
        x = nn.functional.log_softmax(x, dim=2).permute(1, 0, 2)
        return x

In [ ]:
def get_metrics(true_texts, pred_texts):
    numCharErr = 0
    numCharTotal = 0
    numStringOK = 0
    numStringTotal = 0
    word_eds, word_true_lens = [], []
    for i in range(len(pred_texts)):
        pred = pred_texts[i]
        true = true_texts[i]
    
        numStringOK += 1 if true == pred else 0
        numStringTotal += 1
        dist = editdistance.eval(pred, true)
        numCharErr += dist
        numCharTotal += len(true)
        
        pred_words = pred.split()
        true_words = true.split()
        word_eds.append(editdistance.eval(pred_words, true_words))
        word_true_lens.append(len(true_words))
    
    charErrorRate = numCharErr / numCharTotal
    wordErrorRate = sum(word_eds) / sum(word_true_lens)
    stringAccuracy = numStringOK / numStringTotal
    return charErrorRate, wordErrorRate, stringAccuracy


def val_loop(data_loader, model, tokenizer, device):
    model.to(DEVICE)
    all_texts, all_preds = [], []
    for images, texts, _, _ in data_loader:
        batch_size = len(texts)
        text_preds = predict(images, model, tokenizer, device)
        all_texts.append(texts)
        all_preds.append(text_preds)
    all_texts, all_preds = np.concatenate(all_texts), np.concatenate(all_preds)
    avg_cer, wer, acc = get_metrics(all_texts, all_preds)
    print(f'Validation CER: {avg_cer:.4f}  WER: {wer:.4f}  ACCURACY: {acc:.4f}')	 
    return avg_cer


def train_loop(data_loader, model, criterion, optimizer, scheduler, epoch, gradient_accumulation_steps=4):
    optimizer.zero_grad()
    loss_avg = AverageMeter()
    model.to(DEVICE)
    model.train()
    scaled_loss = 0
    for step, (images, texts, enc_pad_texts, text_lens) in tqdm(enumerate(data_loader), total=len(data_loader)):
        images = images.to(DEVICE)
        batch_size = len(texts)

        with autocast('cuda'):
            output = model(images)
            output_lenghts = torch.full(
                size=(output.size(1),),
                fill_value=output.size(0),
                dtype=torch.long)
            loss = criterion(output, enc_pad_texts, output_lenghts, text_lens)
            scaled_loss += loss.item()
            loss_avg.update(loss.item(), batch_size)
            loss.backward()

            if (step + 1) % gradient_accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()
                scaled_loss = 0

    for param_group in optimizer.param_groups:
        lr = param_group['lr']
    print(f'\nEpoch {epoch}, Loss: {loss_avg.avg:.5f}, LR: {lr:.7f}')
    return loss_avg.avg


def predict(images, model, tokenizer, device):
    model.eval()
    images = images.to(device)
    with torch.no_grad():
        output = model(images)
    pred = torch.argmax(output.detach().cpu(), -1).permute(1, 0).numpy()
    text_preds = tokenizer.decode(pred)
    return text_preds


def get_loaders(tokenizer, config):
    train_transforms = get_train_transforms(
        height=config['image']['height'],
        width=config['image']['width'])
    train_loader = get_data_loader(
        json_path=config['train']['json_path'],
        root_path=config['train']['root_path'],
        transforms=train_transforms,
        tokenizer=tokenizer,
        batch_size=config['train']['batch_size'],
        drop_last=True)
    val_transforms = get_val_transforms(
        height=config['image']['height'],
        width=config['image']['width'])
    val_loader = get_data_loader(
        transforms=val_transforms,
        json_path=config['val']['json_path'],
        root_path=config['val']['root_path'],
        tokenizer=tokenizer,
        batch_size=config['val']['batch_size'],
        drop_last=False)
    return train_loader, val_loader


def train(config):
    tokenizer = Tokenizer(config['alphabet'])
    os.makedirs(config['save_dir'], exist_ok=True)
    train_loader, val_loader = get_loaders(tokenizer, config)

    model = CRNN(number_class_symbols=tokenizer.get_num_chars(), pretrained=True)
    model.to(DEVICE)

    criterion = torch.nn.CTCLoss(blank=0, reduction='mean', zero_infinity=True)
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer=optimizer, mode='min', factor=0.3, patience=3)
    loss_best = np.inf
    for epoch in range(100):
        loss_avg = train_loop(train_loader, model, criterion, optimizer, scheduler, epoch, gradient_accumulation_steps=2)
        scheduler.step(loss_avg)

        if loss_avg < loss_best:
            loss_best = loss_avg
            torch.save(model.state_dict(), os.path.join(config['save_dir'], f'recognition_V1.ckpt'))
            print('Model weights saved')

In [ ]:
train(config_json)